# Deep Learning

In [1]:
from keras.layers import Input, Dense, Lambda, Layer
from keras.models import Model
from keras import regularizers
import keras
import pandas as pd
import numpy as np
from keras import backend as K
from keras import metrics
from collections import namedtuple
pd.set_option("display.max_rows",35)
%matplotlib inline

Using TensorFlow backend.


In [2]:
kdd_train_2labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
kdd_test_2labels = pd.read_pickle("dataset/kdd_test_2labels.pkl")

#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels_y.pkl")
#y_train_labels = pd.read_pickle("dataset/kdd_train_2labels.pkl")
#y_test_labels = pd.read_pickle("dataset/kdd_test_2labels_y.pkl")

output_columns_2labels = ['is_Attack','is_Normal']

from sklearn import model_selection as ms
from sklearn import preprocessing as pp

x_input = kdd_train_2labels.drop(output_columns_2labels, axis = 1)
y_output = kdd_train_2labels.loc[:,output_columns_2labels]

ss = pp.StandardScaler()
x_input = ss.fit_transform(x_input)

#le = pp.LabelEncoder()
#y_train = le.fit_transform(y_train_labels).reshape(-1, 1)
#y_test = le.transform(y_test_labels).reshape(-1, 1)

y_train = kdd_train_2labels.loc[:,output_columns_2labels].values

x_train, x_valid, y_train, y_valid = ms.train_test_split(x_input, 
                              y_train, 
                              test_size=0.1)
#x_valid, x_test, y_valid, y_test = ms.train_test_split(x_valid, y_valid, test_size = 0.4)

x_test = kdd_test_2labels.drop(output_columns_2labels, axis = 1)
y_test = kdd_test_2labels.loc[:,output_columns_2labels].values

x_test = ss.transform(x_test)

#x_train = np.hstack((x_train, y_train))
#x_valid = np.hstack((x_valid, y_valid))

#x_test = np.hstack((x_test, np.random.normal(loc = 0, scale = 0.01, size = y_test.shape)))

In [3]:
input_dim = 122
intermediate_dim = 80
latent_dim = 32
batch_size = 1409
epochs = 5
hidden_layers = 8
classes = 2

class Train:
    def train():
        Train.x = Input(shape=(input_dim,))
        
        hidden_encoder = Train.x
        for i in range(hidden_layers):
            hidden_encoder = Dense(intermediate_dim, activation='relu')(hidden_encoder)

        mean_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        logvar_encoder = Dense(latent_dim, activation=None)(hidden_encoder)

        def get_distrib(args):

            mean_encoder, logvar_encoder = args

            # Sample epsilon
            epsilon = np.random.normal(loc=0.0, scale=0.05, size = (batch_size, latent_dim))

            # Sample latent variable
            z = mean_encoder + K.exp(logvar_encoder / 2) * epsilon
            return z

        z = Lambda(get_distrib)([mean_encoder, logvar_encoder])

        hidden_decoder = z
        for i in range(hidden_layers):
            hidden_decoder = Dense(intermediate_dim, activation="relu")(hidden_decoder)

        Train.x_ = Dense(input_dim, activation=None, name='vae_output')(hidden_decoder)
        
        Train.y = Dense(classes, activation='softmax', name='softmax_output')(z)
        

def get_loss(x, x_):
    xent_loss = input_dim * metrics.binary_crossentropy(x, x_) 
    kl_loss = - 0.5 * K.sum(1 + logvar_encoder - K.square(mean_encoder) - K.exp(logvar_encoder), axis=-1)
    
    return K.abs(K.mean(xent_loss + kl_loss + label_loss))



In [4]:
import itertools
#features_arr = [4, 16, 32, 256, 1024]
#hidden_layers_arr = [2, 6, 10, 100]

features_arr = [4, 16, 32]
hidden_layers_arr = [2, 6, 10]

epoch_arr = [10]

score = namedtuple("score", ['epoch', 'no_of_features','hidden_layers','train_score', 'test_score'])
scores = []
predictions = pd.DataFrame()

for e, h, f in itertools.product(epoch_arr, hidden_layers_arr, features_arr):
    
    print(" \n Current Layer Attributes - epochs:{} hidden layers:{} features count:{}".format(e,h,f))
    latent_dim = f
    epochs = e
    hidden_layers = h

    Train.train()

    vae_model = Model(inputs = Train.x, outputs = Train.y)
    vae_model.compile(optimizer = "adam", loss = "mean_squared_error", metrics = ['accuracy'] )

    train_size = x_train.shape[0] - x_train.shape[0]%batch_size
    valid_size = x_valid.shape[0] - x_valid.shape[0]%batch_size

    vae_model.fit(x = x_train[:train_size,:], y = y_train[:train_size,:], 
                  shuffle=True, epochs=epochs, 
                  batch_size = batch_size, 
                  #validation_data = (x_valid[:valid_size,:], y_valid[:valid_size,:]),
                  validation_data = (x_test, y_test),
                  verbose = 1)
    
    score_train = vae_model.evaluate(x_valid[:valid_size,:], y = y_valid[:valid_size,:],
                               batch_size = batch_size,
                               verbose = 1)
    
    score_test = vae_model.evaluate(x_test, y = y_test,
                           batch_size = batch_size,
                           verbose = 1)
    
    y_test_pred = vae_model.predict(x_test, batch_size=batch_size)
    
    y_pred = np.argmax(y_test_pred[:,-2:], axis = 1)
    
    curr_pred = pd.DataFrame({"{}_{}_{}".format(e,f,h):y_pred},)
    predictions = pd.concat([predictions, curr_pred], axis = 1)
    
    scores.append(score(e,f,h,score_train[-1], score_test[-1])) #score_test[-1]))
    
    print("\n Train Acc: {}, Test Acc: {}".format(score_train[-1], 
                                                  score_test[-1])  )
    
scores = pd.DataFrame(scores)


 
 Current Layer Attributes - epochs:10 hidden layers:2 features count:4
Train on 112720 samples, validate on 22544 samples
Epoch 1/10
112720/112720 [==============================] - 0s - loss: 0.0426 - acc: 0.9488 - val_loss: 0.2015 - val_acc: 0.7716
Epoch 2/10
112720/112720 [==============================] - 0s - loss: 0.0152 - acc: 0.9836 - val_loss: 0.2027 - val_acc: 0.7728
Epoch 3/10
112720/112720 [==============================] - 0s - loss: 0.0137 - acc: 0.9853 - val_loss: 0.2074 - val_acc: 0.7685
Epoch 4/10
112720/112720 [==============================] - 0s - loss: 0.0129 - acc: 0.9860 - val_loss: 0.2084 - val_acc: 0.7693
Epoch 5/10
112720/112720 [==============================] - 0s - loss: 0.0120 - acc: 0.9869 - val_loss: 0.1997 - val_acc: 0.7742
Epoch 6/10
112720/112720 [==============================] - 0s - loss: 0.0069 - acc: 0.9924 - val_loss: 0.1935 - val_acc: 0.7914
Epoch 7/10
112720/112720 [==============================] - 0s - loss: 0.0053 - acc: 0.9939 - val_loss

112720/112720 [==============================] - 0s - loss: 0.0133 - acc: 0.9858 - val_loss: 0.2047 - val_acc: 0.7828
Epoch 5/10
112720/112720 [==============================] - 0s - loss: 0.0080 - acc: 0.9907 - val_loss: 0.2039 - val_acc: 0.7825
Epoch 6/10
112720/112720 [==============================] - 0s - loss: 0.0059 - acc: 0.9933 - val_loss: 0.1966 - val_acc: 0.7929
Epoch 7/10
112720/112720 [==============================] - 0s - loss: 0.0059 - acc: 0.9933 - val_loss: 0.2124 - val_acc: 0.7785
Epoch 8/10
112720/112720 [==============================] - 0s - loss: 0.0051 - acc: 0.9939 - val_loss: 0.2047 - val_acc: 0.7826
Epoch 9/10
112720/112720 [==============================] - 0s - loss: 0.0045 - acc: 0.9946 - val_loss: 0.1950 - val_acc: 0.7959
Epoch 10/10
19726/22544 [=========================>....] - ETA: 0s
 Train Acc: 0.9958303719758987, Test Acc: 0.7958658635616302
 
 Current Layer Attributes - epochs:10 hidden layers:10 features count:4
Train on 112720 samples, validate o

In [5]:
scores.sort_values("test_score", ascending=False)

,epoch,no_of_features,hidden_layers,train_score,test_score
8,10,32,10,0.993524,0.803451
2,10,32,2,0.995387,0.800701
5,10,32,6,0.995830,0.795866
1,10,16,2,0.995387,0.793426
0,10,4,2,0.993967,0.787571
6,10,4,10,0.993967,0.780784
3,10,4,6,0.992903,0.779099
4,10,16,6,0.994411,0.778611
7,10,16,10,0.994588,0.775727


In [6]:
predictions.to_pickle("dataset/vae_dense_keras_predictions.pkl")
scores.to_pickle("dataset/vae_dense_keras_scores.pkl")